<a href="https://colab.research.google.com/github/esthel7/ai/blob/master/water_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
pip install urllib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [13]:
# 필요한 라이브러리 불러오기

from selenium import webdriver 
#selenium은 브라우저를 자동화시킴. 자동으로 페이지 열고 실행하도록(사람이 하는 것처럼)
#webdriver는 파이썬이 브라우저를 시작하도록 함

from urllib.request import urlopen ##인터넷 리소스 가져오기
from selenium.webdriver.common.keys import Keys #자동으로 키보드 눌리게하기 위함
import time
import urllib.request #url 가져오기
import os
from selenium.common.exceptions import NoSuchElementException #try, except, if 사용하기
from selenium.common.exceptions import ElementNotVisibleException #페이지 렌더링 되기 전 시간 대기하기
from selenium.webdriver.common.by import By #더 쉬운 함수 사용 위함

In [14]:
#구글 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#크롬 드라이버 사용하기

!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [16]:
#크롬 웹드라이버 연결

chrome_options = webdriver.ChromeOptions() #여기부터 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #여기까지 colab에서 돌리기 위한 코드..? (원래는 주피터)

driver = webdriver.Chrome("chromedriver", chrome_options=chrome_options)
driver.get("https://www.google.co.kr/imghp?hl=ko&ogbl") # 구글 이미지 검색 url

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [17]:
#검색어 입력하기

search = "침수"
elem = driver.find_element(By.CSS_SELECTOR, '[name="q"]') #css 찾기
elem.send_keys(search) #검색창에 검색할 내용(name)넣기
elem.send_keys(Keys.RETURN) #검색할 내용을 넣고 enter를 치기

In [18]:
#스크롤 끝까지 내리기 - 스크롤을 내리면 새로운 브라우저가 갱신되기 때문

SCROLL_PAUSE_TIME = 1

last_height = driver.execute_script("return document.body.scrollHeight") #브라우저의 높이를 자바스크립트로 찾음

while True:
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 브라우저 끝까지 스크롤을 내림
  time.sleep(SCROLL_PAUSE_TIME) #페이지 로드되기 기다리기
  new_height = driver.execute_script("return document.body.scrollHeight") #새 브라우저 높이 계산

  if new_height == last_height:
    try:
      driver.find_element_by_css_selector(".mye4qd").click() #스크롤을 내리다가 "결과 더보기" 뜨면 클릭
    except: #더이상 내릴 스크롤이 없음
      break
  last_height = new_height

In [19]:
#이미지 찾아서 원본 파일로 저장하기

images = driver.find_elements(By.CSS_SELECTOR,".rg_i.Q4LuWd") #작게 뜬 이미지들 모두 선택(elements)
count = 1

for image in images:
  if count>400: #400개 이상 크롤링하면 정확도가 떨어짐
    break
  try:
    image.click()
    time.sleep(2)
    imgUrl = driver.find_element(By.CSS_SELECTOR,'#Sva75c > div > div > div.dFMRD > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div.nIWXKc.JgfpDb.cZEg1e > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id.b0vFpe > div > a > img').get_attribute("src")
    # 크게 뜬 이미지 선택하여 Copy selector로 받아온 경로를 설정

    print(imgUrl)
    
    urllib.request.urlretrieve(imgUrl, "/content/drive/MyDrive/goodde_image/" + search + "_" + str(count) + ".jpg")
    # goodde_image 라는 폴더는 구글 드라이브 제일 루트 위치에

    print("Image saved: {}_{}.jpg".format(search, count)) #{}에 들어갈 수를 format함수를 통해 전달
    count += 1
  except:
    pass

driver.close()

https://img.hankyung.com/photo/202208/01.30877354.1.jpg
Image saved: 침수_1.jpg
https://images.chosun.com/resizer/FEWiO52xH80ukuXRUxlmxXF2ElY=/616x0/smart/cloudfront-ap-northeast-1.images.arcpublishing.com/chosun/7NFKWPITH7HGLH2445CJ2ST2UE.jpg
Image saved: 침수_2.jpg
https://www.socialfocus.co.kr/news/photo/202008/8328_13098_5138.jpg
Image saved: 침수_3.jpg
https://img.khan.co.kr/news/2022/08/08/news-p.v1.20220808.8d0fc8f80d8140ada1c571f5376d0dab_P1.jpg
Image saved: 침수_4.jpg
https://newsimg.sedaily.com/2020/08/07/1Z6HJTPI7L_1.jpg
Image saved: 침수_5.jpg
